# Toronto Neighborhood Clustering - IBM Data Science Capstone
#### by fil.coutinho

## 1. Dataframe Preparation

In [12]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [13]:
df_toronto.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)

#Drop 'Not Assigned' rows
df_toronto.drop(df_toronto.loc[df_toronto['Borough']=='Not assigned'].index, inplace=True)
df_toronto.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [14]:
count_postal = df_toronto.groupby(['PostalCode']).count()
count_postal.head()

,Borough,Neighborhood
PostalCode,,
M1B,1,1
M1C,1,1
M1E,1,1
M1G,1,1
M1H,1,1


In [15]:
#Check for duplicate postal codes
count_postal.loc[count_postal['Neighborhood'] > 1]

,Borough,Neighborhood
PostalCode,,


In [16]:
# Check for postal code with Neighborhood not assigned but Borough existent
df_toronto.loc[df_toronto['Neighborhood']=='Not assigned']

,PostalCode,Borough,Neighborhood


In [17]:
df_toronto.shape

(103, 3)

## 2. Dataframe Preparation

In [ ]:
# !pip install pgeocode

In [18]:
import pgeocode

nomi = pgeocode.Nominatim('ca')
nomi.query_postal_code("M4A")['latitude']
nomi.query_postal_code("M4A")['longitude']

ModuleNotFoundError: No module named 'pgeocode'

In [19]:
df_toronto['Latitude'] = 0
df_toronto['Longitude'] = 0

df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,0,0
3,M4A,North York,Victoria Village,0,0
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0
5,M6A,North York,"Lawrence Manor, Lawrence Heights",0,0
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",0,0


In [20]:

body = client_61bf4a96af25446cb70fbd99af7c40a1.get_object(Bucket='ibmdatasciencecapstone-donotdelete-pr-butz1ybluhym17',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_geo = pd.read_csv(body)
df_geo.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [22]:
df_geo.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)
df_toronto_geo = pd.merge(df_toronto, df_geo, on='PostalCode')
df_toronto_geo.head()

,PostalCode,Borough,Neighborhood,Latitude_x,Longitude_x,Latitude_y,Longitude_y
0,M3A,North York,Parkwoods,0,0,43.753259,-79.329656
1,M4A,North York,Victoria Village,0,0,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",0,0,43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",0,0,43.662301,-79.389494
